In [ ]:
import pandas as pd
import numpy as np
import re
from itertools import groupby
from os import listdir, path

In [ ]:
def formata_data_hora(mi):
    #troca / por - e remove 'UTC' das strings de hora
    data, hora = mi[0].replace('/', '-'), mi[1].replace(' UTC', '')
    if len(hora) == 4:  hora = hora[:2] + ':' + hora[2:]
    return data, hora

def ler_arquivo(arquivo):
    #lendo o arquivo em duas partes
    metadado = pd.read_csv(arquivo, encoding='latin_1', sep=';', decimal=',', nrows=8, header=None).set_index(0)
    dados = pd.read_csv(arquivo, skiprows=8, encoding='latin_1', sep=';', decimal=',').replace(-9999, np.nan)
    
    #removendo o : dos nomes dos metadados e normalizando as que variam com o tempo
    metadado.index = ['REGIÃO', 'UF', 'ESTAÇÃO', 'CODIGO (WMO)', 'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'DATA DE FUNDAÇÃO (YYYY-MM-DD)']
    metadado = metadado.T.iloc[0]
    
    #acrescentando os dados de altitude, latitude e longitudo no dataframe (podem mudar de ano a ano)
    dados.assign(**{
        'ALTITUDE': metadado['ALTITUDE'],
        'LATITUDE': metadado['LATITUDE'],
        'LONGITUDE': metadado['LONGITUDE']
    })
    
    #normalizando nomes de colunas que mudam na planilha
    dados.rename({
        dados.columns[0]: 'Data',
        dados.columns[1]: 'Hora',
        dados.columns[6]: 'RADIACAO GLOBAL (KJ/m²)'
    }, axis='columns', inplace=True)
    
    #retornando o codigo da estaçao, dicionario com a regiao, uf e nome da estação, dataframe com todos os dados
    return metadado['CODIGO (WMO)'], metadado[['REGIÃO', 'UF', 'ESTAÇÃO']].to_dict(), dados.set_index(list(dados.columns[:2]))

def concat_years(code, file_list, output_dir):
    #faz a leitura de cada arquivo
    years = []
    for file in sorted(file_list):
        _, metadata, dataframe = ler_arquivo(file)
        years.append(dataframe)
    
    #concatena e arruma o index
    df = pd.concat(years, copy=False)
    df.index = df.index.map(formata_data_hora)
        
    #salva o arquivo concatenado e retorna os metadados
    name = f'{metadata["REGIÃO"]}_{metadata["UF"]}_{code}_{metadata["ESTAÇÃO"]}'
    df.sort_index().to_csv(f'{output_dir}/{name}.csv', sep=';', decimal=',', encoding='latin_1')

def padronize_data(inmet_dir, output_dir):
    #salvando o nome de todas as planilhas
    arquivos = []
    for folder in listdir(inmet_dir):
        c = f'{inmet_dir}/{folder}'
        if not path.isdir(c): continue
        if path.isdir(f'{c}/{folder}'): c = f'{c}/{folder}'
        arquivos += [f'{c}/{a}' for a in listdir(c) if a.endswith('.CSV')]
    print(f'Total de arquivos: {len(arquivos)}')
    
    #para cada codigo, concatena e adiciona o codigo e salva os metadados
    metadatas = []
    search_groups = lambda s: re.search('_([A-Z][0-9]{3})_', s).group(1)
    for k, grupo in groupby(sorted(arquivos, key=search_groups), search_groups):
        print(f'Codigo {k}...', end='')
        concat_years(k, grupo, output_dir)
        print('OK')

In [ ]:
path_inmet = 'datasets/inmet'
path_output = 'datasets/agregados'
padronize_data(path_inmet, path_output)

In [ ]:
#todo: adaptar para os novos formatos
def resumo(arquivo):
    metadado = pd.read_csv(arquivo, encoding='latin_1', sep=';', decimal=',', nrows=8, header=None)
    local = f'{metadado.iloc[0, 1]} -- {metadado.iloc[1, 1]} -- {metadado.iloc[3,1]} -- {metadado.iloc[2,1]}'
    
    dados = pd.read_csv(arquivo, skiprows=8, encoding='latin_1', sep=';', decimal=',').replace(-9999, np.nan)
    return dados.min().rename(local), dados.max().rename(local), (dados.isnull().sum(axis=0)/len(dados)*100).rename(local), dados.apply(lambda x: x.groupby(x.notna().cumsum()).cumcount().max()).rename(local)

def resumo_anual(ano):
    tabelas = {
        'min': [], #minimo de cada coluna
        'max': [], #maximo de cada coluna
        'porcentagem': [], #porcentagem de nulos em cada coluna
        'sequencia': []  #maior sequencia de nulos
    }
    
    d = f'{p}/{ano}/'
    if path.isdir(f'{d}{ano}'): d = f'{d}{ano}/'
        
    for file in listdir(d):
        minimo, maximo, porcentagem, sequencia = resumo(d+file)
        tabelas['min'].append(minimo)
        tabelas['max'].append(maximo)
        tabelas['porcentagem'].append(porcentagem)
        tabelas['sequencia'].append(sequencia)
    
    for k, v in tabelas.items():
        df = pd.DataFrame(v)
        df.columns = [f'{k.upper()} - {c}' for c in df.columns]
        tabelas[k] = df
        
    colunas = [v.columns for k, v in tabelas.items()]
    colunas = [j for i in zip(*colunas) for j in i]
    return pd.concat(tabelas.values(), axis=1).reindex(columns=colunas).sort_index()#

In [ ]:
todos = {}
for ano in range(2000, 2022):
    print(f'Procesando ano {ano}...', end='')
    df = resumo_anual(ano)
    todos[ano] = df
    #df.to_csv(f'{ano}.csv', sep=';', decimal=',', encoding='latin_1')
    print('OK')

In [ ]:
filtrado = [(k, v[v.index.str.startswith(('S -- SC', 'SE -- SP'))]) for k, v in todos.items()]
pd.DataFrame([v['PORCENTAGEM - PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].rename(k) for k, v in filtrado if not v.empty]).T.sort_index().iloc[:, 14:].applymap(lambda x: 'x' if x < 1/13*100 else '').to_csv('porcentagem.csv', sep=';', decimal=',', encoding='latin_1')